# Overview

**Inputs:**
 - Emissions or global mean temperature (GMT) scenarios from IAM.  
 - Climate impacts maps by Global Warming Level.  

**Outputs:**
 - Maps of climate impacts through time, per scenario.

Steps  
1. Import climate impacts database. NetCDF 4-d file with different impacts as variables.
2. Import the scenarios to be run. Can be global mean temp scenarios or Emissions|CO2 scenarios.
3. Preprocessing steps.
4. Run.


In [1]:
## Add figure here

In [2]:
import dask
import glob
import numpy as np
import os
import pyam
import time
import xarray as xr
import yaml
from dask import delayed
from dask.distributed import Client
from process_config import *
from rime_functions import *

with open(yaml_path, "r") as f:
    params = yaml.full_load(f)

<IPython.core.display.Javascript object>

In [7]:
yaml_path

'C:\\Github\\climate_impacts_processing\\hotspots.yml'

In [3]:
dask.config.set(scheduler="processes")
dask.config.set(num_workers=num_workers)
client = Client()


C:\Users\byers\Anaconda3\envs\py310\lib\site-packages\distributed\node.py:179: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 60903 instead
  warnings.warn(


Open http://localhost:8787/status

In [4]:
### Import the climate impacts database files


In [6]:
params

{'protocol': {'cdd': '3b',
  'dri_qtot': '2b',
  'dri': '2b',
  'heatwave': '3b',
  'ia_var': '2b',
  'ia_var_qtot': '2b',
  'lc': '3b',
  'precip': '3b',
  'seas': '2b',
  'seas_qtot': '2b',
  'sdd': '3b',
  'sdd_24p0': '3b',
  'sdd_20p0': '3b',
  'sdd_18p3': '3b',
  'tr20': '3b',
  'yields_cereals': '3b',
  'yields_sugarc': '3b',
  'yields_oilseeds': '3b',
  'yields_roots': '3b',
  'yields_food': '3b',
  'wsi': '2b'},
 'indicators': {'cdd': {'cdd': {'short_name': 'cdd',
    'long_name': 'Consecutive dry days',
    'unit': 'days/yr',
    'ind_min': 0,
    'ind_max': 180,
    'ind_cmap': 'YlOrRd',
    'diff': 'relative',
    'diff_min': -20,
    'diff_max': 20,
    'diff_cmap': 'RdBu_r',
    'score_range': [24, 13, 8, 0]}},
  'dri': {'drought_intensity': {'short_name': 'dri',
    'long_name': 'Drought intensity',
    'unit': '-',
    'ind_min': 0,
    'ind_max': 1730,
    'ind_cmap': 'gist_earth',
    'diff': 'relative',
    'diff_min': -20,
    'diff_max': 20,
    'diff_cmap': 'RdBu_r

### Import scenarios data

Load an IAMC scenarios dataset.  
Decide whether using global mean temperature or CO2 mode.  
Assign SSPs if missing and fix duplicate temperatures.  


In [5]:
df_scens_in = pyam.IamDataFrame(fname_input_scenarios)
dft = df_scens_in.filter(variable=temp_variable)
# dft = np.round(dft.as_pandas()[pyam.IAMC_IDX + ["year", "value", "Ssp_family"]], 2)
# dft = pyam.IamDataFrame(dft)
# Replace & fill missing SSP scenario allocation
dft = ssp_helper(dft, ssp_meta_col="Ssp_family", default_ssp="SSP2")


pyam - INFO: Running in a notebook, setting up a basic logging at level INFO
pyam.core - INFO: Reading file C:\Users\byers\IIASA\ECE.prog - Documents\Research Theme - NEXUS\Hotspots_Explorer_2p0\rcre_testing\testing_2\emissions_temp_AR6_small.xlsx
pyam.core - INFO: Reading meta indicators


NameError: name 'ssp_helper' is not defined

In [ ]:
dft

## Example: multiple IAM scenarios, 1 climate indicator

In [7]:
print("Test multiple scenarios, 1 indicator")
start = time.time()

ind = "precip"
var = "sdii"
short = params["indicators"][ind][var]["short_name"]

ssp = "ssp2"



files = glob.glob(os.path.join(impact_data_dir, ind, f"*{short}_{ssp}*{ftype}.nc4"))
mapdata = xr.open_mfdataset(
    files, preprocess=remove_ssp_from_ds, combine="nested", concat_dim="gmt"
)


map_out_MS = map_transform_gmt_multi_dask(
    dft.filter(model="POLES ADVANCE"),
    mapdata,
    years,
    use_dask=True,
    gmt_name="threshold",
    interpolation=interpolation,
)

comp = dict(zlib=True, complevel=5)
encoding = {var: comp for var in map_out_MS.data_vars}
filename = f"{output_folder_maps}scenario_maps_multiscenario_{ftype}_test_notebook.nc"
map_out_MS.to_netcdf(filename, encoding=encoding)

print("FINISHED Test multiple scenarios, 1 indicator")
print(f"{time.time()-start}")

Test multiple scenarios, 1 indicator
Single indicator mode
POLES_ADVANCE_ADVANCE_2020_1.5C-2100
POLES_ADVANCE_ADVANCE_2020_Med2C
POLES_ADVANCE_ADVANCE_2020_WB2C
POLES_ADVANCE_ADVANCE_2030_1.5C-2100
POLES_ADVANCE_ADVANCE_2030_Med2C
POLES_ADVANCE_ADVANCE_2030_Price1.5C
POLES_ADVANCE_ADVANCE_2030_WB2C
POLES_ADVANCE_ADVANCE_INDC
POLES_ADVANCE_ADVANCE_NoPolicy
POLES_ADVANCE_ADVANCE_Reference
FINISHED Test multiple scenarios, 1 indicator
10.552809476852417


In [8]:
map_out_MS

<xarray.Dataset>
Dimensions:                           (lat: 360, lon: 720, year: 9)
Coordinates:
  * lon                               (lon) int64 0 1 2 3 4 ... 716 717 718 719
  * lat                               (lat) int64 0 1 2 3 4 ... 356 357 358 359
  * year                              (year) int32 2015 2025 2035 ... 2085 2095
Data variables:
    POLES_ADVANCE_ADVANCE_2020_Med2C  (lat, lon, year) float64 nan nan ... nan
    POLES_ADVANCE_ADVANCE_2020_WB2C   (lat, lon, year) float64 nan nan ... nan
    POLES_ADVANCE_ADVANCE_2030_Med2C  (lat, lon, year) float64 nan nan ... nan
    POLES_ADVANCE_ADVANCE_INDC        (lat, lon, year) float64 nan nan ... nan
    POLES_ADVANCE_ADVANCE_Reference   (lat, lon, year) float64 nan nan ... nan

## Example: 1 scenario, multiple indicators (faster with dask=False)


In [9]:

print("Test 1 scenario, multiple indicators")
start = time.time()
ssp = "ssp2"
mapdata = xr.Dataset()
indicators = [
    "cdd",
    "precip",
    "dri",
    "dri_qtot",
    "ia_var",
    "ia_var_qtot",
    "sdd_18p3",
    "sdd_24p0",
]  #'heatwave']

for ind in indicators:
    print(ind)
    for var in params["indicators"][ind]:
        short = params["indicators"][ind][var]["short_name"]
        files = glob.glob(
            os.path.join(impact_data_dir, ind, f"*{short}_{ssp}*{ftype}.nc4")
        )
        mapdata[short] = xr.open_mfdataset(
            files, preprocess=remove_ssp_from_ds, combine="nested", concat_dim="gmt"
        )[short]



Test 1 scenario, multiple indicators
cdd
precip
dri
dri_qtot
ia_var
ia_var_qtot
sdd_18p3
sdd_24p0


In [10]:
mapdata

<xarray.Dataset>
Dimensions:     (lat: 360, lon: 720, gmt: 6)
Coordinates:
    stats       object 'mean'
  * lat         (lat) float64 89.75 89.25 88.75 88.25 ... -88.75 -89.25 -89.75
  * lon         (lon) float64 -179.8 -179.2 -178.8 -178.2 ... 178.8 179.2 179.8
    threshold   (gmt) float64 1.2 1.5 2.0 2.5 3.0 3.5
    quantile    float64 0.95
Dimensions without coordinates: gmt
Data variables:
    cdd         (gmt, lat, lon) float64 dask.array<chunksize=(1, 360, 720), meta=np.ndarray>
    pr_r10      (gmt, lat, lon) float64 dask.array<chunksize=(1, 360, 720), meta=np.ndarray>
    pr_r20      (gmt, lat, lon) float64 dask.array<chunksize=(1, 360, 720), meta=np.ndarray>
    pr_r95p     (gmt, lat, lon) float64 dask.array<chunksize=(1, 360, 720), meta=np.ndarray>
    pr_r99p     (gmt, lat, lon) float64 dask.array<chunksize=(1, 360, 720), meta=np.ndarray>
    sdii        (gmt, lat, lon) float64 dask.array<chunksize=(1, 360, 720), meta=np.ndarray>
    dri         (gmt, lat, lon) float64 dask.array<chunksize=(1, 360, 720), meta=np.ndarray>
    dri_qtot    (gmt, lat, lon) float64 dask.array<chunksize=(1, 360, 720), meta=np.ndarray>
    iavar       (gmt, lat, lon) float64 dask.array<chunksize=(1, 360, 720), meta=np.ndarray>
    iavar_qtot  (gmt, lat, lon) float64 dask.array<chunksize=(1, 360, 720), meta=np.ndarray>
    sdd_c_18p3  (gmt, lat, lon) float64 dask.array<chunksize=(1, 360, 720), meta=np.ndarray>
    sdd_c_24p0  (gmt, lat, lon) float64 dask.array<chunksize=(1, 360, 720), meta=np.ndarray>

In [11]:
map_out_MI = map_transform_gmt_multi_dask(
    dft.filter(model="AIM*", scenario="SSP1-34"),
    mapdata,
    years,
    use_dask=False,
    gmt_name="threshold",
)

comp = dict(zlib=True, complevel=5)
encoding = {var: comp for var in map_out_MI.data_vars}
filename = f"{output_folder_maps}scenario_maps_multiindicator_{ftype}_test_notebook.nc"
# map_out_MI.to_netcdf(filename, encoding=encoding)

print("FINISHED 1 scenario, multiple indicators")
print(f"{time.time()-start}")


Single scenario mode
cdd
Warning! Min temperature below 1.2°C  1.12, data before not possible
pr_r10
Warning! Min temperature below 1.2°C  1.12, data before not possible
pr_r20
Warning! Min temperature below 1.2°C  1.12, data before not possible
pr_r95p
Warning! Min temperature below 1.2°C  1.12, data before not possible
pr_r99p
Warning! Min temperature below 1.2°C  1.12, data before not possible
sdii
Warning! Min temperature below 1.2°C  1.12, data before not possible
dri
Warning! Min temperature below 1.2°C  1.12, data before not possible
dri_qtot
Warning! Min temperature below 1.2°C  1.12, data before not possible
iavar
Warning! Min temperature below 1.2°C  1.12, data before not possible
iavar_qtot
Warning! Min temperature below 1.2°C  1.12, data before not possible
sdd_c_18p3
Warning! Min temperature below 1.2°C  1.12, data before not possible
sdd_c_24p0
Warning! Min temperature below 1.2°C  1.12, data before not possible
FINISHED 1 scenario, multiple indicators
15.842880964279175


In [12]:
map_out_MI

<xarray.Dataset>
Dimensions:     (lat: 360, lon: 720, year: 9)
Coordinates:
  * lon         (lon) int64 0 1 2 3 4 5 6 7 ... 712 713 714 715 716 717 718 719
  * lat         (lat) int64 0 1 2 3 4 5 6 7 ... 352 353 354 355 356 357 358 359
  * year        (year) int32 2015 2025 2035 2045 2055 2065 2075 2085 2095
Data variables:
    cdd         (lat, lon, year) float64 nan nan nan nan nan ... nan nan nan nan
    pr_r10      (lat, lon, year) float64 nan nan nan nan nan ... nan nan nan nan
    pr_r20      (lat, lon, year) float64 nan nan nan nan nan ... nan nan nan nan
    pr_r95p     (lat, lon, year) float64 nan nan nan nan nan ... nan nan nan nan
    pr_r99p     (lat, lon, year) float64 nan nan nan nan nan ... nan nan nan nan
    sdii        (lat, lon, year) float64 nan nan nan nan nan ... nan nan nan nan
    dri         (lat, lon, year) float64 nan nan nan nan nan ... nan nan nan nan
    dri_qtot    (lat, lon, year) float64 nan nan nan nan nan ... nan nan nan nan
    iavar       (lat, lon, year) float64 nan nan nan nan nan ... nan nan nan nan
    iavar_qtot  (lat, lon, year) float64 nan nan nan nan nan ... nan nan nan nan
    sdd_c_18p3  (lat, lon, year) float64 nan nan nan nan nan ... nan nan nan nan
    sdd_c_24p0  (lat, lon, year) float64 nan nan nan nan nan ... nan nan nan nan

## Plot dashboard

In [13]:
from rime_functions import plot_maps_dashboard

In [14]:
plot_maps_dashboard(map_out_MI, filename=None, indicators=['dri','dri_qtot',], year=2055, cmap='magma_r', shared_axes=True, clim=None)

Exception: filename must be string and end with '.html'

In [ ]:
ds = map_out_MI
year=2055
model = 'AIM'
scenario = "SSP1-34"
# if indicators==None:
#     indicators = list(ds.data_vars)
# elif isinstance(indicators, list):
#     if all(x in ds.data_vars for x in indicators)==False:
#         raise Exception(f"Error: not all items in indicators were found in ds.")
# elif isinstance(indicators, list)==False:
#     raise Exception(f"Error: indicators must be of type list.")


# Subset the dataset. Check dims and length

ds = check_ds_dims(ds)


if 'year' in ds.dims:
    ds = ds.sel(year=year).squeeze()
elif len(ds.dims) != 2:
    raise Exception(f"Error: Year not a dimension and more than 2 dimensions in dataset")

plot_list = []
# Run loop through indicators (variables)
indicators = ['dri','dri_qtot']
for i in indicators:

    new_plot = ds[i].hvplot(x='lon', y='lat', cmap='magma_r', shared_axes=True)
    plot_list = plot_list + [new_plot]

plot = hv.Layout(plot_list).cols(3)

filename = f'maps_dashboard_{model}_{scenario}.html'


# Plot - check filename
# if type(filename) is None:
#     filename = 'maps_dashboard_{model}_{scenario}.html'

# elif (type(filename) is str):
#     if (filename[:-5]) != '.html':
#         raise Exception(f"filename {filename} must end with '.html'")

# else:
#     raise Exception(f"filename must be string and end with '.html'")



hvplot.save(plot, filename)

In [ ]:
import holoviews as hv
import hvplot.pandas
import hvplot.xarray